# 1.3 Web Search Integration

In this hands-on lab, you'll enhance an AI agent from a knowledge-limited chatbot to an intelligent assistant capable of accessing real-time web information. You'll see firsthand how tool integration transforms agent capabilities.

By the end of this lab, you will be able to:
- Build a basic LangChain agent using Azure OpenAI
- Understand the limitations of LLMs with static training data
- Integrate external web search tools to enable real-time information retrieval
- Compare agent behavior with and without tool augmentation
- Debug and trace agent reasoning using LangSmith



In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Step 1: Load Environment Configuration

### 📖 What You're Learning

In production environments, **never hardcode API keys** in your code! This step demonstrates the industry-standard practice of using environment variables for sensitive credentials.

### 🔑 Key Concepts
- **Environment Variables**: Store sensitive data outside your codebase
- **python-dotenv**: Automatically loads variables from `.env` files
- **Security Best Practice**: Separates configuration from code for better security and flexibility

### ✅ Action Required
Run the cell below to load your Azure OpenAI and Tavily API credentials from the `.env` file.

**Expected Output:** `True` (indicating successful load)

In [2]:
import os
from langchain.agents import create_agent
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

agent = create_agent(
    model=model
)

## Step 2: Create Your First Basic Agent

### 📖 What You're Learning

Here you'll build a foundational LangChain agent powered by Azure OpenAI. This agent has **NO external tools** - it can only use knowledge from its training data.

### 🔑 Key Concepts
- **AzureChatOpenAI**: LangChain's wrapper for Azure-hosted OpenAI models
- **Agent Architecture**: A reasoning engine that can plan and execute tasks
- **Model Configuration**: Uses environment variables for deployment flexibility
- **Tool-Free Agent**: Limited to static knowledge with no real-time data access

### 🎓 Understanding the Code
- `create_agent()` initializes a ReAct-style agent
- The agent receives only the `model` parameter - no tools attached yet
- It will rely solely on training data knowledge (typically with a cutoff date)

### ✅ Action Required
Execute this cell to create your baseline agent. We'll test its limitations next!

In [3]:
from langchain.messages import HumanMessage

question = HumanMessage(content="How up to date is your training knowledge?")

response = agent.invoke(
    {"messages": [question]}
)

## Step 3: Test the Basic Agent's Knowledge Limitations

### 📖 What You're Learning

Let's expose the fundamental limitation of LLMs: **their training data has a cutoff date**. This means they can't answer questions about recent events without external tools.

### 🔑 Key Concepts
- **HumanMessage**: LangChain's standardized format for user input
- **Agent Invocation**: How to send messages and receive responses
- **Knowledge Cutoff**: The date beyond which the model has no information
- **Response Structure**: Agents return a dictionary with message history

### 🎓 What to Expect
The agent will honestly tell you when its knowledge ends (e.g., "October 2023"). It cannot provide information about events after that date.

### ✅ Action Required
Run this cell and observe how the agent acknowledges its training limitations.

In [4]:
print(response['messages'][-1].content)

My training knowledge is current up until **October 2023**. Any information or events after that date are not part of my knowledge base.


## Step 4: Display the Agent's Response

### 📖 What You're Learning

Understanding how to extract meaningful output from agent responses is crucial for building real applications.

### 🔑 Key Concepts
- **Response Navigation**: `response['messages'][-1]` accesses the final agent message
- **Content Extraction**: `.content` retrieves the actual text
- **Message Array**: The response contains the full conversation history

### ✅ Action Required
Run this cell to see the agent's honest admission about its knowledge cutoff date.

💡 **Training Tip**: Keep this response in mind - you'll compare it with the enhanced agent's capabilities later!

## Add web search tool

In [7]:
from tavily import TavilyClient
client = TavilyClient()
response = client.search(
    query="What are the latest updates from NVIDIA ?"
)
print(response)

{'query': 'What are the latest updates from NVIDIA ?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.networkworld.com/article/3562856/nvidia-latest-news-and-insights.html', 'title': 'Nvidia: Latest news and insights | Network World', 'content': 'August 12, 2025: Nvidia announced two new professional GPUs, the RTX Pro 4000 Small Form Factor (SFF) and the RTX Pro 2000. Built on its Blackwell architecture,', 'score': 0.73023266, 'raw_content': None}, {'url': 'https://www.nvidia.com/en-in/drivers/nvidia-update/', 'title': 'NVIDIA Update', 'content': 'NVIDIA Update keeps your PC up-to-date with the latest NVIDIA drivers by notifying you when a new driver is available and directing you to the driver on www.', 'score': 0.7251239, 'raw_content': None}, {'url': 'https://www.nvidia.com/en-us/software/nvidia-app/release-highlights/', 'title': 'Updates and Release Highlights - NVIDIA', 'content': "What's new in NVIDIA App 11.0.2 · DLSS Multi Frame Gene

## Step 5: Explore the Tavily Web Search API

### 📖 What You're Learning

Before integrating web search into your agent, let's test Tavily directly. This helps you understand what data the search API returns and how it works.

### 🔑 Key Concepts
- **Tavily Search API**: An AI-optimized search engine designed for LLM applications
- **Direct API Testing**: Always test external services independently first
- **Real-Time Information**: Unlike the agent, this can fetch current web content
- **Structured Results**: Returns JSON with titles, URLs, snippets, and relevance scores

### 🎓 What to Expect
You'll see a JSON response containing:
- `results`: Array of search findings with URLs and content
- `answer`: AI-generated summary of the search results
- `images`: Related images if available

### ✅ Action Required
Execute this cell to search for recent NVIDIA news. Notice how you get **current information**!

💡 **Training Tip**: Compare this real-time data with what the basic agent could provide.

In [8]:
!echo $TAVILY_API_KEY

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


## Step 6: Verify Your API Key Setup

### 📖 What You're Learning

Debugging tip: **Always verify your environment variables are loaded correctly** before troubleshooting deeper issues.

### 🔑 Key Concepts
- **Shell Commands in Notebooks**: The `!` prefix executes bash/zsh commands
- **Environment Variable Verification**: Confirms the API key is accessible
- **Debugging Best Practice**: Check configuration before assuming code errors

### ✅ Action Required
Run this cell to confirm your Tavily API key is properly loaded.

**Expected Output:** Your Tavily API key string (e.g., `tvly-xxxxxxxxxxxx`)

⚠️ **Troubleshooting**: If you see an empty output, check that `TAVILY_API_KEY` is defined in your `.env` file.

In [6]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

web_search.invoke("Who is the current mayor of San Francisco?")

{'query': 'Who is the current mayor of San Francisco?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Mayor_of_San_Francisco',
   'title': 'Mayor of San Francisco - Wikipedia',
   'content': 'The current mayor is Democrat Daniel Lurie.',
   'score': 0.9994253,
   'raw_content': None},
  {'url': 'https://apnews.com/article/san-francisco-new-mayor-liberal-city-81ea0a7b37af6cbb68aea7ef5cc6a4f0',
   'title': "San Francisco's new mayor is starting to unite the fractured city",
   'content': 'San Francisco Mayor Daniel Lurie, a political newcomer and Levi Strauss heir, has marked his first 100 days with a hands-on, business-friendly approach.',
   'score': 0.9993538,
   'raw_content': None},
  {'url': 'https://www.sf.gov/departments--office-mayor',
   'title': 'Office of the Mayor - SF.gov',
   'content': 'Daniel Lurie is the 46th Mayor of the City and County of San Francisco.',
   'score': 0.99620515,
   'raw_content': None

## Step 7: Create a LangChain Tool for Web Search

### 📖 What You're Learning

This is where the magic happens! You'll **transform the Tavily API into a LangChain tool** that your agent can autonomously use.

### 🔑 Key Concepts
- **@tool Decorator**: Converts any Python function into a LangChain-compatible tool
- **Type Annotations**: Helps the agent understand input/output types (`str` → `Dict[str, Any]`)
- **Docstring as Tool Description**: The agent reads "Search the web for information" to decide when to use this tool
- **Tool Testing**: Always test tools independently before giving them to agents
- **Abstraction Layer**: Wraps external APIs in a standard interface

### 🎓 Understanding the Magic
When you add this tool to an agent:
1. The agent reads the docstring to understand the tool's purpose
2. It examines the type annotations to know what inputs are required
3. It autonomously decides **when** to call the tool based on user questions
4. It processes the returned data and synthesizes a human-readable answer

### ✅ Action Required
Execute this cell to create and test your web search tool.

**Expected Output:** Search results about the current San Francisco mayor

💡 **Training Tip**: Notice how the function is immediately tested with `.invoke()` - this ensures it works before integration!

In [9]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

agent = create_agent(
    model=model,
    tools=[web_search]
)

question = HumanMessage(content="Who is the current mayor of San Francisco?")

response = agent.invoke(
    {"messages": [question]}
)

## Step 8: Build the Enhanced Agent with Web Search

### 📖 What You're Learning

Now you're creating a **tool-augmented agent** - the same Azure OpenAI model, but with superpowers! It can now access real-time information from the web.

### 🔑 Key Concepts
- **Tool-Augmented Agent**: Adds `tools=[web_search]` parameter to enable tool use
- **Autonomous Decision Making**: The agent decides when to use the tool (you don't tell it!)
- **Multi-Step Reasoning**: The agent can plan: "I need current info → use web_search → synthesize answer"
- **Same Model, New Capabilities**: Identical LLM but now has access to current information
- **Real-World Testing**: Asking about the current mayor requires today's knowledge

### 🎓 How the Agent Thinks
1. **Receives question**: "Who is the current mayor of San Francisco?"
2. **Reasoning**: "I need current information about a political position"
3. **Decision**: "I should use the web_search tool"
4. **Execution**: Calls `web_search("current mayor San Francisco")`
5. **Synthesis**: Reads search results and formulates a natural language answer

### ✅ Action Required
Run this cell to create your enhanced agent and watch it autonomously use web search!

💡 **Training Tip**: The agent wasn't explicitly told to search the web - it figured that out on its own by reading the tool's docstring!

In [10]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='Who is the current mayor of San Francisco?', additional_kwargs={}, response_metadata={}, id='13e75f27-0a3f-48b8-a82c-e4625d6ce82d'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 52, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-Cr2Ax5MKms5iYFxmqw3ZH9LN3XKS3', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'sa

## Step 9: Inspect the Agent's Reasoning Process

### 📖 What You're Learning

**Peek behind the curtain!** This cell reveals the complete thought process and tool calls your agent made.

### 🔑 Key Concepts
- **Message Chain**: The full conversation including internal reasoning
- **Tool Call Tracing**: See exactly when and how the agent called `web_search`
- **Tool Results**: The raw data returned from the web search
- **Response Synthesis**: How the agent transformed search results into a natural answer
- **Debugging Power**: Essential for understanding agent behavior in production

### 🎓 What You'll See
The message array contains multiple entries:
1. **HumanMessage**: Your original question
2. **AIMessage with tool_calls**: Agent decides to use the web_search tool
3. **ToolMessage**: Results returned from web search
4. **AIMessage**: Final synthesized answer for the user

### ✅ Action Required
Execute this cell to see the complete agent reasoning trace.

💡 **Training Tip**: In production systems, understanding this message flow is crucial for debugging why an agent made specific decisions!

trace: https://smith.langchain.com/public/59432173-0dd6-49e8-9964-b16be6048426/r

## 🔍 LangSmith Tracing Link

### 📖 What This Is

**LangSmith** is a powerful observability platform for LLM applications. The link above provides a visual trace of your agent's execution.

### 💡 Pro Tip
Click the trace link to see:
- Token usage and costs
- Latency for each step
- Visual representation of the agent's decision tree
- Input/output for every tool call

This is invaluable for optimizing performance and debugging complex agents in production!

In [11]:
print(response['messages'][-1].content)

The current mayor of San Francisco is Daniel Lurie.


## Step 10: Extract and Compare the Final Answer

### 📖 What You're Learning

Let's get the clean, user-facing response and compare it with your basic agent's limitations!

### 🔑 Key Concepts
- **Clean Output**: Extracts only the final answer, hiding internal tool calls
- **Information Synthesis**: The agent combined web search results into coherent prose
- **Real-Time Accuracy**: This answer reflects current information, not outdated training data
- **User Experience**: This is what your end users would see in a production application

### ✅ Action Required
Run this cell to see the agent's final answer about the current mayor.

### 🎓 Lab Completion Reflection

**Compare your results:**
- **Basic Agent** (Step 4): Could only report its knowledge cutoff date
- **Enhanced Agent** (Step 10): Provided current, accurate information by using web search

**Key Takeaway:** Tool integration transforms agents from static knowledge bases into dynamic, real-time information assistants!

---

## 🎉 Congratulations!

You've successfully:
- ✅ Built a basic LangChain agent with Azure OpenAI
- ✅ Identified limitations of static training data
- ✅ Integrated Tavily web search as a LangChain tool
- ✅ Created a tool-augmented agent that autonomously decides when to search
- ✅ Debugged and traced agent reasoning processes

### 🚀 Next Steps
Try these challenges to deepen your learning:
1. Ask the agent about other recent events (e.g., "What are the latest AI announcements?")
2. Modify the tool's docstring and observe how it affects agent behavior
3. Add error handling to the web_search tool
4. Create a second tool (e.g., calculator) and give your agent multiple capabilities